In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
df_jobs=pd.read_csv('/kaggle/input/job-titles/job-titles.txt')

In [3]:
df_jobs.rename(columns={'1st grade teacher':'name'},inplace=True)

In [4]:
df_jobs['Target']=0

In [5]:
df_jobs.head(10)

,name,Target
0,1st pressman,0
1,1st pressman on web press,0
2,21 dealer,0
3,2nd grade teacher,0
4,2nd pressman,0
5,2 year olds preschool teacher,0
6,3d animator,0
7,3d artist,0
8,3d designer,0
9,3d modeler,0


In [6]:
len(df_jobs)

73379

**importing the names dataset**

In [4]:
names=pd.read_csv('/kaggle/input/indian-names-dataset/Dataset-Indian-Names-master/Indian-Male-Names.csv')
femalenames=pd.read_csv('/kaggle/input/indian-names-dataset/Dataset-Indian-Names-master/Indian-Female-Names.csv')
# names=pd.read_csv('/kaggle/input/indian-names-dataset/Dataset-Indian-Names-master/Indian_Names.csv')
newnames=pd.concat([names,femalenames])

In [5]:
newnames.drop(columns=['gender','race'],inplace=True,axis=1)

In [9]:

newnames.head()

,name
0,barjraj
1,ramdin verma
2,sharat chandran
3,birender mandal
4,amit


In [6]:
newnames['Target']=0

In [7]:
newnames.head()

,name,Target
0,barjraj,0
1,ramdin verma,0
2,sharat chandran,0
3,birender mandal,0
4,amit,0


In [8]:
places=pd.read_csv('/kaggle/input/all-india-pincode-directory-aug21/All India Pincode Directory Aug-21.csv')
places['OfficeName']

/tmp/ipykernel_28/2394976996.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  places=pd.read_csv('/kaggle/input/all-india-pincode-directory-aug21/All India Pincode Directory Aug-21.csv')


0              Peddakotla B.O
1              Pinnadhari B.O
2         Yerraguntapalle B.O
3         Obulareddipalli B.O
4              Odulapalli B.O
                 ...         
157121     Rly Road Meerut SO
157122             SGMandi SO
157123            W K Road SO
157124             Kakkoti SO
157125           Kotuvalli SO
Name: OfficeName, Length: 157126, dtype: object

In [9]:
rem=['CircleName', 'RegionName', 'DivisionName',  'Pincode',
       'OfficeType', 'Delivery', 'District', 'StateName', 'Latitude',
       'Longitude']
places.drop(columns=rem,axis=1,inplace=True)

In [10]:
places['OfficeName']=places['OfficeName'].str[0:-2]

In [11]:
places.loc[places['OfficeName'].str[-1]=='B']['OfficeName']=places['OfficeName'].str[0:-1]

/tmp/ipykernel_28/157462596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  places.loc[places['OfficeName'].str[-1]=='B']['OfficeName']=places['OfficeName'].str[0:-1]


In [12]:
places['Target']=0

In [13]:
places.rename(columns={'OfficeName':'name'},inplace=True)

In [14]:
print(df_jobs.columns,places.columns,newnames.columns)

Index(['name'], dtype='object') Index(['name', 'Target'], dtype='object') Index(['name', 'Target'], dtype='object')


In [15]:

import pandas as pd
df_companies=pd.read_csv('/kaggle/input/all-indian-companies-registration-data-1900-2019/registered_companies.csv',low_memory=False)

In [16]:
df_companies.drop(columns=['CORPORATE_IDENTIFICATION_NUMBER', 'COMPANY_STATUS',
       'COMPANY_CLASS', 'COMPANY_CATEGORY', 'COMPANY_SUB_CATEGORY',
       'DATE_OF_REGISTRATION', 'REGISTERED_STATE', 'AUTHORIZED_CAP',
       'PAIDUP_CAPITAL', 'INDUSTRIAL_CLASS',
       'PRINCIPAL_BUSINESS_ACTIVITY_AS_PER_CIN', 'REGISTERED_OFFICE_ADDRESS',
       'REGISTRAR_OF_COMPANIES', 'EMAIL_ADDR', 'LATEST_YEAR_ANNUAL_RETURN',
       'LATEST_YEAR_FINANCIAL_STATEMENT'],axis=1,inplace=True)

In [17]:
df_companies.rename(columns={'COMPANY_NAME':'name'},inplace=True)

In [18]:
df_companies['Target']=1

In [19]:
df=pd.concat([newnames,places,df_jobs,df_companies])

In [20]:
df.Target.unique()

array([ 0., nan,  1.])

In [22]:
df.isna().sum()

name         55
Target    73379
dtype: int64

In [23]:
df.dropna(inplace=True)

In [24]:
df.isna().sum()

name      0
Target    0
dtype: int64

In [25]:
X=df['name']
y=df['Target']
x_train,x_test,y_train,y_test=train_test_split(X,y)

In [26]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout

# Sample data
words = x_train
labels = y_train

# Word2Vec
tokenizer = Tokenizer()
tokenizer.fit_on_texts(words)
sequences = tokenizer.texts_to_sequences(words)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Padding sequences
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# LSTM RNN
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length))
model.add(Dropout(0.2))
model.add(LSTM(32))

model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Convert labels to numpy array
labels = np.array(labels)

# Train the model
model.fit(padded_sequences, labels, epochs=10, verbose=1)

# Test the model
test_words = x_test
test_sequences = tokenizer.texts_to_sequences(test_words)
test_padded_sequences = pad_sequences(test_sequences, maxlen=max_length)
predictions = model.predict(test_padded_sequences)



Epoch 1/10
51082/51082 [==============================] - 388s 7ms/step - loss: 0.0033 - accuracy: 0.9991
Epoch 2/10
51082/51082 [==============================] - 353s 7ms/step - loss: 8.0943e-04 - accuracy: 0.9998
Epoch 3/10
51082/51082 [==============================] - 342s 7ms/step - loss: 5.3650e-04 - accuracy: 0.9998
Epoch 4/10
51082/51082 [==============================] - 329s 6ms/step - loss: 4.2315e-04 - accuracy: 0.9999
Epoch 5/10
51082/51082 [==============================] - 322s 6ms/step - loss: 3.3511e-04 - accuracy: 0.9999
Epoch 6/10
51082/51082 [==============================] - 319s 6ms/step - loss: 2.7353e-04 - accuracy: 0.9999
Epoch 7/10
51082/51082 [==============================] - 321s 6ms/step - loss: 2.3113e-04 - accuracy: 0.9999
Epoch 8/10
51082/51082 [==============================] - 325s 6ms/step - loss: 2.2309e-04 - accuracy: 0.9999
Epoch 9/10
51082/51082 [==============================] - 320s 6ms/step - loss: 1.8146e-04 - accuracy: 1.0000
Epoch 10/10
17

In [30]:
ans=[]
for pred in predictions:
    if pred>0.5:
        ans.append(1)
    else:
        ans.append(0)
        

In [31]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, ans)

In [32]:
accuracy

0.9821497723297612

In [33]:
# After training the model


model_path = '/kaggle/working/companyname.h5'

model.save(model_path)

print("Model saved successfully.")


Model saved successfully.


In [34]:
import pickle

# saving
with open('companynametokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
max_length

19